In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Deep Neural Networks 
## Session 11c: lecture

## Working with Tensorflow 
- Two hidden layer
- ${relu}$ activation function
- Adam
- Multi-class output
- FIFA dataset

</p><img src='../../prasami_images/prasami_color_tutorials_small.png' width='400' alt="By Pramod Sharma : pramod.sharma@prasami.com" align="left"/>

In [ ]:
# Lets import some libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

%matplotlib inline

In [ ]:
# Some basic parameters

inpDir = '../../input' # location where input data is stored
outDir = '../output' # location to store outputs
modelDir = '../models'
subDir = 'fifa_2019'
altName = 'relu_adam'

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production

np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results
 
tf.random.set_seed(RANDOM_STATE) # setting for TensorFlow as well


EPOCHS = 100 # number of epochs
ALPHA = 0.1 # learning rate

BATCH_SIZE = 256 # other sizes leave last batch partial

TRAIN_SIZE = int(BATCH_SIZE*64) # to create 64 batches of 256 rows each

PATIENCE = 20
LR_FACTOR = 0.5
LR_PATIENCE = 10

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (15,15),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

CMAP = plt.cm.brg

plt.rcParams.update(params) # update rcParams

### Settings so that Tensorflow can not Hog all the GPU memory

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

def fn_plot_tf_hist(hist_df : pd.DataFrame):
    '''
    Args:
      hist_df : pandas Dataframe with four columns
                For 'x' values, we will use index
    '''
    fig, axes = plt.subplots(1,2 , figsize = (15,6))

    # properties  matplotlib.patch.Patch 
    props = dict(boxstyle='round', facecolor='aqua', alpha=0.4)
    facecolor = 'cyan'
    fontsize=12
    
    # Get columns by index to eliminate any column naming error
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[1]
    y3 = hist_df.columns[2]
    y4 = hist_df.columns[3]

    # Where was min loss
    best = hist_df[hist_df[y3] == hist_df[y3].min()]
    
    ax = axes[0]

    hist_df.plot(y = [y1,y3], ax = ax, colormap=CMAP)


    # little beautification
    txtFmt = "Loss: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y3]) #text to plot
    
    # place a text box in upper middle
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y3].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y3].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y3].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid();
    ax.legend(loc = 'upper left') # model legend to upper left

    ax = axes[1]

    hist_df.plot( y = [y2, y4], ax = ax, colormap=CMAP)
    
    # little beautification
    txtFmt = "Accuracy: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y2],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in upper middle
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y4].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow
    
    
    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y2.capitalize())
    ax.grid()
    ax.legend(loc = 'lower left')
    
    plt.tight_layout()

## Read FIFA 2019 data set

In [ ]:
data_df = pd.read_csv(os.path.join(inpDir, 'fifa_2019.csv'))
data_df.shape

## Investigate and prepare features

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>
     <ol>
         <li>Display unique values in the columns data_df[col].unique()</li>
         <li>Remove rows with 'position' = null (data_df = data_df[data_df["Position"].notnull()])</li>
</ol> 
</div>

In [ ]:
# Display unique values in the columns

In [ ]:
# removing rows with position = null


In [ ]:
# Following columns appear to be relevant for our analysis
rel_cols = ["Position", 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
            'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
            'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
            'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
            'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
            'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
            'GKKicking', 'GKPositioning', 'GKReflexes']

data_df = data_df[rel_cols]
data_df.head()

In [ ]:
# Are there NaNs Still?

data_df.isna().sum()

In [ ]:
goalkeeper = 'GK'
forward = ['ST', 'LW', 'RW', 'LF', 'RF', 'RS','LS', 'CF']
midfielder = ['CM','RCM','LCM', 'CDM','RDM','LDM', 'CAM', 'LAM', 'RAM', 'RM', 'LM']
defender = ['CB', 'RCB', 'LCB', 'LWB', 'RWB', 'LB', 'RB']

### Assemble "Position" column.
> data_df.loc[data_df["Position"] == "GK", "Position"] = 0
> data_df.loc[data_df["Position"].isin(defender), "Position"] = 1
> data_df.loc[data_df["Position"].isin(midfielder), "Position"] = 2
> data_df.loc[data_df["Position"].isin(forward), "Position"] = 3
> data_df['Position'] = pd.to_numeric(data_df['Position'], downcast="integer")

In [ ]:
#Assign labels to goalkeepers

#Defenders

#Midfielders

#Forward

# Convert Column "Position" to numeric so that Pandas does not complain


In [ ]:
labels = {0: 'Goal Keeper', 1: 'Defender', 2: 'Mid-Fielder', 3: 'Forward'}

In [ ]:
y = data_df["Position"].values

X = data_df.drop("Position", axis = 1)

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>Lets try with one hot encoding
</div>

>y = pd.get_dummies(y).values

In [ ]:
# one hot encoding


In [ ]:
#  Split the data in training and test sets to measure performance of the model.
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=TRAIN_SIZE,
                                                    stratify=y,
                                                    random_state=RANDOM_STATE )

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
(33-4)/3

## Prepare model
### Functional API

<p class="alert alert-block alert-info" style="font-family: Arial; font-size:1.2em">You can also instantiate a Model with the <strong>"Functional API"</strong>, where you start from Input, you chain layer calls to specify the model's forward pass, and finally you create your model from inputs and outputs:

> inputs = tf.keras.Input(shape=(33,))
> x2 = tf.keras.layers.Dense(23, activation=tf.nn.relu)(inputs)
> x3 = tf.keras.layers.Dense(12, activation=tf.nn.relu)(x2)
> outputs = tf.keras.layers.Dense(4)(x3)
> model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# Using Model class



<p class="alert alert-block alert-info" style="font-family: Arial; font-size:1.2em;color:black;">
    Always a good idea to Unit Test
</p>

In [ ]:
X_train[:1].shape

In [ ]:
predictions = model(X_train[:1]).numpy()
predictions

In [ ]:
predictions.sum()

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
tf.nn.softmax(predictions).numpy()

In [ ]:
tf.nn.softmax(predictions).numpy().sum()

In [ ]:
model.summary()

## pass model

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>add all callback `callbacks=[model_callback, early_stopping, reduce_lr]''
</div>
> checkpoint_dir = os.path.join(modelDir, subDir)

> checkpoint_prefix = os.path.join(checkpoint_dir, altName)

> model_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                    monitor='val_loss',
                                                    mode='auto',
                                                    save_weights_only=True,
                                                    save_best_only=True,
                                                    verbose=1)

> early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    verbose=1,
    restore_best_weights=True
)

> reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=LR_FACTOR,
    patience=LR_PATIENCE,
    verbose=1
)
> loss_fn = tf.keras.losses.CategoricalCrossentropy ( from_logits = True)
> model.compile( optimizer= 'adam', loss = loss_fn,
              metrics=['accuracy']
             )

In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy ( from_logits = True)
model.compile( optimizer= 'adam', 
              loss = loss_fn,
              metrics=['accuracy']
             )

In [ ]:
history = model.fit(X_train, y_train,
                   validation_data=[X_test, y_test],
                   epochs=EPOCHS,
                   callbacks=[model_callback, early_stopping, reduce_lr])

In [ ]:
hist_df = pd.DataFrame(history.history)
fn_plot_tf_hist(hist_df)

In [ ]:
model.evaluate ( X_test,  y_test, verbose=2)

## Predictions

In [ ]:
model_output = model.output
softmax_outputs = tf.keras.layers.Softmax()(model_output)

probability_model = tf.keras.Model(inputs=model.input, outputs=softmax_outputs)

In [ ]:
y_pred = probability_model(X_train).numpy()
y_pred

In [ ]:
print(classification_report(np.argmax(y_train, axis = 1), np.argmax(y_pred, axis = 1)))

In [ ]:
y_pred = probability_model(X_test).numpy()

print('Accuracy score on Test Data :{:.5f}'.format(accuracy_score(np.argmax(y_test, axis = 1), 
                                                                  np.argmax(y_pred, axis = 1))))

In [ ]:
print(classification_report(np.argmax(y_test, axis = 1), 
                            np.argmax(y_pred, axis = 1)))

In [ ]:
cm  = confusion_matrix(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=labels.values())

fig, ax = plt.subplots(figsize = (4,4))

disp.plot(ax = ax, cmap = 'Blues', xticks_rotation = 'vertical', colorbar=False)

plt.show();

## Assignment A03: Increase accuracy of this model
Hint: change activation, add layers, dropout and batch normalization, etc...